# Домашнее задание по деревьям принятия решений
Задача кредитного скоринга - одна из наиболее популярных областей, где применяются алгоритмы машинного обучения.<br>
Здесь мы будет прогнозировать, что человек просрочит выплаты по кредиту на 3 месяца и более (целевой признак - Delinquent90).<br>
В качестве метрики была выбрана AUC (Area Under Curve).

Признаки клиентов банка:
- Age - возраст (вещественный)
- Income - месячный доход (вещественный)
- BalanceToCreditLimit - отношение баланса на кредитной карте к лимиту по кредиту (вещественный)
- DIR - Debt-to-income Ratio (вещественный)
- NumLoans - число заемов и кредитных линий
- NumRealEstateLoans - число ипотек и заемов, связанных с недвижимостью (натуральное число)
- NumDependents - число членов семьи, которых содержит клиент, исключая самого клиента (натуральное число)
- Num30-59Delinquencies - число просрочек выплат по кредиту от 30 до 59 дней (натуральное число)
- Num60-89Delinquencies - число просрочек выплат по кредиту от 60 до 89 дней (натуральное число)
- Delinquent90 - были ли просрочки выплат по кредиту более 90 дней (бинарный) - имеется только в обучающей выборке

## 1. Обучение дерева принятия решений
## 1.1 Подгрузка библиотек и инициализация вспомогательных функций

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score, roc_curve

# функция, выдающая базовые метрики классификации
def quality_report(prediction, actual):
    print("Accuracy: {:.3f}\nPrecision: {:.3f}\nRecall: {:.3f}\nf1_score: {:.3f}".format(
        accuracy_score(prediction, actual),
        precision_score(prediction, actual),
        recall_score(prediction, actual),
        f1_score(prediction, actual)
    ))

# функция для отрисовки roc-кривой и подсчёта
def plot_roc_curve(prob_prediction, actual):
    fpr, tpr, thresholds = roc_curve(y_val, prob_prediction)
    auc_score = roc_auc_score(y_val, prob_prediction)

    plt.plot(fpr, tpr, label='ROC curve ')
    plt.plot([0, 1], [0, 1])
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC AUC: {:.3f}'.format(auc_score))
    plt.show()

## 1.2 Загрузка данных

In [ ]:
train_df = pd.read_csv('credit_scoring_train.csv', index_col='client_id')
test_df = pd.read_csv('credit_scoring_test.csv', index_col='client_id')

In [ ]:
# размер тренировочного набора
train_df.shape

(75000, 10)

In [ ]:
# бегло вглянем на данные в тренировочном наборе
train_df.head()

,DIR,Age,NumLoans,NumRealEstateLoans,NumDependents,Num30-59Delinquencies,Num60-89Delinquencies,Income,BalanceToCreditLimit,Delinquent90
client_id,,,,,,,,,,
0,0.496289,49.1,13,0,0.0,2,0,5298.360639,0.387028,0
1,0.433567,48.0,9,2,2.0,1,0,6008.056256,0.234679,0
2,2206.731199,55.5,21,1,NaN,1,0,NaN,0.348227,0
3,886.132793,55.3,3,0,0.0,0,0,NaN,0.971930,0
4,0.000000,52.3,1,0,0.0,0,0,2504.613105,1.004350,0


В данных наблюдаются пропуски. Необходимо посчитать для каждого признака, сколько информации утеряно, и принять решение о методе обработки пропущенных значений.

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75000 entries, 0 to 74999
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   DIR                    75000 non-null  float64
 1   Age                    75000 non-null  float64
 2   NumLoans               75000 non-null  int64  
 3   NumRealEstateLoans     75000 non-null  int64  
 4   NumDependents          73084 non-null  float64
 5   Num30-59Delinquencies  75000 non-null  int64  
 6   Num60-89Delinquencies  75000 non-null  int64  
 7   Income                 60153 non-null  float64
 8   BalanceToCreditLimit   75000 non-null  float64
 9   Delinquent90           75000 non-null  int64  
dtypes: float64(5), int64(5)
memory usage: 6.3 MB


Атрибуты таблицы с тренировочными данными представлены в численном виде, обработка типов для признаков не требуется.

## 1.3 Предобработка данных
### 1.3.1 Обработка пропусков и невалидных значений

In [ ]:
# посмотрим, сколько всего пропусков в каждом атрибуте в %
train_df.isnull().sum() / train_df.shape[0] * 100

DIR                       0.000000
Age                       0.000000
NumLoans                  0.000000
NumRealEstateLoans        0.000000
NumDependents             2.554667
Num30-59Delinquencies     0.000000
Num60-89Delinquencies     0.000000
Income                   19.796000
BalanceToCreditLimit      0.000000
Delinquent90              0.000000
dtype: float64

In [ ]:
test_df.isnull().sum() / test_df.shape[0]

DIR                      0.000000
Age                      0.000000
NumLoans                 0.000000
NumRealEstateLoans       0.000000
NumDependents            0.025547
Num30-59Delinquencies    0.000000
Num60-89Delinquencies    0.000000
Income                   0.197960
BalanceToCreditLimit     0.000000
Delinquent90             0.000000
dtype: float64

В тренировочном наборе наблюдаются невалидные значения:
- 2.5% в атрибуте NumDependents;
- 19.7% в атрибуте Income

В тестовом наборе также присутствуют пропущенные значения:
- менее 1% в атрибуте NumDependents
- менее 1% в атрибуте Income

Записи с невалидными значениями можно было бы удалить, но, на мой взгляд, удаление будет критичным, поскольку у нас всего 75_000 записей в тренировочном наборе. После разделения на тренировочный и валидационный датасеты их станет ещё меньше.

Также пропуски можно было бы заполнить одним значением, но в атрибуте Income потеря информации составляет около 20% ($\frac{1}{5}$ ), есть риск, что этим действием я внесу лишние зависимости в данные. <b>На основе этих выводов,</b> я принимаю решение заполнить пропуски в данных с помощью алгоритма машинного обучения через модуль nona.

In [ ]:
from nona.nona import nona
nona(train_df)

ModuleNotFoundError: ignored

In [ ]:
train_df

In [ ]:
train_df.info()

In [ ]:
train_df.isnull().sum() / train_df.shape[0] * 100

Невалидные значения, как показано выше, устранены.

### 1.3.2 Обработка выбросов
Последний штрих $-$ поиск и обработка аномальных значений в данных. Проверим все признаки с помощью ящиков с усами.

In [ ]:
for column in train_df.columns:
    # будем сразу рассчитывать статистически "правильные" max и min
    IQR = np.quantile(train_df[column], .75) - np.quantile(train_df[column], .25)
    stats_max, stats_min = 1.5 * IQR + np.quantile(train_df[column], .75), \
            np.quantile(train_df[column], .25) - 1.5 * IQR

    plt.boxplot(train_df[column])
    plt.title(f"{column}\nmax: {round(stats_max, 2)}, min: {round(stats_min, 2)}")
    plt.show();

Попробуем проанализировать результаты. Однозначно нет выбросов в признаке Delinquent90, поскольку он бинарный.
В атрибутах, названия которых начинаются с Num, присутствуют отрицательные значения, хотя по всем показателям это должны быть натуральные числа.
Посмотрим, сколько аномальных значений среди признаков.

In [ ]:
for column in train_df.columns:
    IQR = np.quantile(train_df[column], .75) - np.quantile(train_df[column], .25)
    stats_max, stats_min = 1.5 * IQR + np.quantile(train_df[column], .75), \
            np.quantile(train_df[column], .25) - 1.5 * IQR

    print(f"{column}\nбольшие: {len(train_df[train_df[column] > stats_max])}; маленькие: {len(train_df[train_df[column] < stats_min])}\n")

Значения признака age не кажутся такими большими и отклоняющимися сильно, но наблюдается сильный разброс в признаке Income. В этом признаке присутствуют отрицательные значения, вероятно, это валидные значения, которые получаются в тех случаях, когда клиент не имеет дохода, но вынужден тратить. Возможно, такие значения в признаке будут однозначно указывать, что клиент не сможет заплатить через 3 месяца. Однако от слишком высоких значений можно избавиться, к тому же их не так много, ведь основная масса клиентов банка имеет средний доход.

In [ ]:
train_df.drop(index=train_df[train_df['Income'] > 13_414].index, inplace=True)
print(train_df.shape)

Также очевидно, что в признаке NumDependents нужно избавиться от записей, в которых значение более 7: очевидно, что количество людей на содержании в среднем 5-6.

In [ ]:
train_df.drop(index=train_df[train_df['NumDependents'] >= 7].index, inplace=True)
print(train_df.shape)

In [ ]:
train_df.columns

В признаках, отвечающих за количество рассрочек и займов у клиента, нужно избавиться от значений, которые больше 20.

In [ ]:
train_df.drop(index=train_df[train_df['Num30-59Delinquencies'] >= 20].index, inplace=True)
train_df.drop(index=train_df[train_df['Num60-89Delinquencies'] >= 20].index, inplace=True)
train_df.drop(index=train_df[train_df['NumLoans'] >= 20].index, inplace=True)
print(train_df.shape)

Также очень странно выглядит признак DIR, уберём в нём выбросы.

In [ ]:
train_df.drop(index=train_df[train_df['DIR'] > 1.89].index, inplace=True)
print(train_df.shape)

In [ ]:
for column in train_df.columns:
    # будем сразу рассчитывать статистически "правильные" max и min
    IQR = np.quantile(train_df[column], .75) - np.quantile(train_df[column], .25)
    stats_max, stats_min = 1.5 * IQR + np.quantile(train_df[column], .75), \
            np.quantile(train_df[column], .25) - 1.5 * IQR

    plt.boxplot(train_df[column])
    plt.title(f"{column}\nmax: {round(stats_max, 2)}, min: {round(stats_min, 2)}")
    plt.show();

## Вывод
Данные предобработаны. В ходе предобработки данных были заполнены пропуски в данных, а также обработаны аномальные значения. Данные представлены в численном виде, они готовы для следующего этапа.

In [ ]:
train_df.head(5)

## 1.4 Обучение модели дерева принятия решений
### 1.4.1 Разделение данных на тренировочный и валиадационный наборы

In [ ]:
# посмотреть на распределение целевой переменной
train_df['Delinquent90'].hist(bins=20, color='green');

В данных явно наблюдается дисбаланс классов. Чтобы гарантировать, что относительные частоты классов приблизительно сохраняются в каждом цикле обучения и цикле проверки, необходимо установить параметр stratify у функции train_test_split. Разделим данные.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    train_df.drop(['Delinquent90'], axis=1),
    train_df['Delinquent90'],
    test_size=0.3,
    random_state=2023,
    stratify=train_df['Delinquent90']
)

In [ ]:
y_train.value_counts(normalize=True)

In [ ]:
y_val.value_counts(normalize=True)

In [ ]:
X_train.hist(figsize=(15, 10));

### 1.4.2 Baseline
Наше первая модель будет наивной и основываться на том предположении, что клиенты банка по целевому признаку будут случайно распределяться с тем же соотношением, какое мы наблюдали в тренировочной и валидационной выборках. Посмотрим и оценим эту гипотезу.

In [ ]:
y_naive = np.random.choice([0, 1], size=y_val.shape[0], p=y_train.value_counts(normalize=True))

In [ ]:
quality_report(y_naive, y_val)

In [ ]:
plot_roc_curve(y_naive, y_val)

Несмотря на то, что правильность такой модели очень высокая, фактически наша модель $-$ модель угадывания, к какому классу относится конкретный клиент банка, о чём и сообщает целевая метрика ROC AUC.

### 1.4.3 Дерево решений без настройки параметров

In [ ]:
first_tree = DecisionTreeClassifier(max_depth=3, random_state=42)
first_tree.fit(X_train, y_train)

In [ ]:
print("Train quality")
quality_report(first_tree.predict(X_train), y_train)

print("\nTest quality")
quality_report(first_tree.predict(X_val), y_val)

In [ ]:
plot_roc_curve(first_tree.predict_proba(X_val)[:, 1], y_val)

Результат гораздо лучше, классификатор на основе дерева принятия решений дал показатель ROC AUC в 78%, при этом я только глубину дерева урезал до 3 уровней.

Модель дерева принятия решений ещё прекрасна тем, что может показать, какие признаки были важны для неё. Визуализируем их.

In [ ]:
featureImportance = pd.DataFrame({"feature": X_train.columns,
                                  "importance": first_tree.feature_importances_})

featureImportance.set_index('feature', inplace=True)
featureImportance.sort_values(["importance"], ascending=False, inplace=True)
featureImportance["importance"].plot(kind='bar');

In [ ]:
# нарисуем получившееся дерево
plot_tree(first_tree,
         feature_names=X_train.columns,
          rounded=True, filled=True);

### 1.4.4 Дерево решений с настройкой параметров с помощью GridSearch
Получившаяся модель хороша, но её можно попробовать улучшить. Потюним параметры.

In [ ]:
tree_params = {
               'max_depth': list(range(3,11)),
               'min_samples_leaf': list(range(3,11)),
               'class_weight': [None, 'balanced']
}

locally_best_tree = GridSearchCV(DecisionTreeClassifier(random_state=42),
                                 tree_params,
                                 verbose=True, n_jobs=-1, cv=5,
                                scoring='roc_auc')
locally_best_tree.fit(X_train, y_train)

In [ ]:
locally_best_tree.best_params_, round(locally_best_tree.best_score_, 3)

In [ ]:
quality_report(locally_best_tree.predict(X_val), y_val)

In [ ]:
plot_roc_curve(locally_best_tree.predict_proba(X_val)[:, 1], y_val)

In [ ]:
featureImportance = pd.DataFrame({"feature": X_train.columns,
                                  "importance": locally_best_tree.best_estimator_.feature_importances_})

featureImportance.set_index('feature', inplace=True)
featureImportance.sort_values(["importance"], ascending=False, inplace=True)
featureImportance["importance"].plot(kind='bar');

In [ ]:
plot_tree(locally_best_tree.best_estimator_,
         rounded=True,
         filled=True);

In [ ]:
cm = confusion_matrix(y_val, locally_best_tree.predict(X_val))
conf_matrix = pd.DataFrame(data = cm, columns = ['Predicted:0','Predicted:1'], index=['Actual:0','Actual:1'])
plt.figure(figsize = (5,5))
sns.heatmap(conf_matrix, annot=True,fmt='d',cmap="YlGnBu", cbar=False);

## 2. Вырастим лес

1. Генерирую 1 000 поднаборов обучающего набора, каждый из которых содержит 100 случайных образцов.

In [ ]:
from sklearn.model_selection import ShuffleSplit

# генерирую 1 000 поднаборов обучающего набора, каждый из которых содержит 100 случайных образцов
n_trees = 1000
n_instances = 100

mini_sets = []

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42)
for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train.iloc[mini_train_index]
    y_mini_train = y_train.iloc[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

In [ ]:
from sklearn.base import clone

# создаю 1 000 деревьев с наилучшими значениями гиперпараметров
forest = [clone(locally_best_tree.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []

# прохожу по каждому из деревьев и даю ему свой поднабор данных
for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)

    y_pred = tree.predict(X_val) # вырабатываю прогноз
    accuracy_scores.append(accuracy_score(y_val, y_pred))

# средняя оценка правильности 1 000 деревьев
np.mean(accuracy_scores)

In [ ]:
Y_pred = np.empty([n_trees, len(X_val)], dtype=np.uint8)
# собрать предсказания с 1 000 деревьев
for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_val)

In [ ]:
from scipy.stats import mode
# найти самое частое предсказание
y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

In [ ]:
quality_report(y_val, y_pred_majority_votes.reshape([-1]))

In [ ]:
plot_roc_curve(y_val, y_pred_majority_votes.reshape([-1]))